# Kapitel 6: Höhere Magie – Deskriptoren, Metaklassen & Review

**Heutige Lernziele:**

Wir werden die wahre Natur von `@property` ergründen und sogar lernen, wie man Klassen zur Laufzeit dynamisch erzeugt und modifiziert. Am Ende fassen wir alles zusammen und bereiten uns auf die Prüfung vor.

* **Das Deskriptor-Protokoll:** Wir demystifizieren `@property`, indem wir die Low-Level-Mechanik dahinter (`__get__`, `__set__`) selbst implementieren.
* **Metaprogrammierung & Metaklassen:** Wir lernen das fortgeschrittenste Konzept der OOP in Python kennen: Klassen, die andere Klassen erzeugen.
* **Gesamtwiederholung & Prüfungssimulation:** Wir festigen das Wissen der gesamten Woche mit komplexen, prüfungsnahen Aufgaben.

---
---

## **6.1: Tiefer Einblick: Das Deskriptor-Protokoll – Die Magie hinter `@property`**

### **Deskriptoren im Detail**

Wir haben in Kapitel 3 die Frage gestellt: "Wie funktioniert `@property` wirklich?". Die Antwort ist eines der mächtigsten, aber oft verborgenen Konzepte in Python: das **Deskriptor-Protokoll**.

**Was ist ein Deskriptor?**
Ein Deskriptor ist **jedes Objekt (also eine Instanz einer Klasse), das mindestens eine der folgenden speziellen Methoden implementiert:**
* `__get__(self, instance, owner)`
* `__set__(self, instance, value)`
* `__delete__(self, instance)`

**Wie funktioniert der Mechanismus?**
Der entscheidende Punkt ist, dass Deskriptoren als **Klassenattribute** in einer anderen Klasse (der "owner class") definiert werden.

Wenn Python auf ein Attribut zugreift (z.B. `mein_obj.mein_attribut`), passiert mehr als nur eine einfache Suche:
1. Python prüft zuerst die Klasse von `mein_obj` (`type(mein_obj)`).
2. Es schaut nach, ob das Attribut `mein_attribut` in der Klasse selbst ein Deskriptor-Objekt ist (also ob es eine `__get__`- oder `__set__`-Methode hat).
3. **Wenn ja,** wird der normale Attribut-Zugriff unterbrochen. Stattdessen ruft Python die entsprechende Methode des Deskriptors auf.

   * Bei Lesezugriff (`x = mein_obj.mein_attribut`): `Deskriptor.__get__(mein_obj, type(mein_obj))`
   * Bei Schreibzugriff (`mein_obj.mein_attribut = wert`): `Deskriptor.__set__(mein_obj, wert)`

**Parameter der Deskriptor-Methoden:**
* `self`: Die Instanz des Deskriptors selbst.
* `instance`: Die Instanz der Klasse, über die auf das Attribut zugegriffen wird (hier: `mein_obj`).
* `owner`: Die Klasse, der der Deskriptor gehört (hier: `type(mein_obj)`).

**Data vs. Non-Data Descriptors:**
Dies ist ein wichtiger Unterschied, der die Zugriffsreihenfolge beeinflusst:
* **Data Descriptor:** Ein Deskriptor, der sowohl `__get__` als auch `__set__` implementiert. Data Descriptors haben die **höchste Priorität** und überschreiben sogar ein gleichnamiges Attribut im `__dict__` der Instanz.
* **Non-Data Descriptor:** Ein Deskriptor, der nur `__get__` implementiert. Er hat eine **niedrigere Priorität**. Wenn eine Instanz in ihrem `__dict__` ein Attribut mit demselben Namen hat, wird dieses bevorzugt und der Deskriptor wird nicht aufgerufen.

`@property` ist nichts anderes als eine einfache, in C implementierte Python-Klasse, die dieses Protokoll für uns umsetzt und die Handhabung extrem vereinfacht.

---

### **Beispiel A : Ein `LoggingDescriptor`**

Wir bauen einen Deskriptor, dessen einzige Aufgabe es ist, jeden Lese- und Schreibzugriff auf ein Attribut zu protokollieren.


In [2]:
class LoggingDescriptor:
    def __init__(self, name):
        self.name = name

    def __get__(self, instance, owner):
        print(f"[LOG] Lesezugriff auf '{self.name}'...")
        # Hole den Wert aus dem __dict__ der Instanz
        return instance.__dict__.get(self.name)

    def __set__(self, instance, value):
        print(f"[LOG] Schreibzugriff auf '{self.name}' mit Wert '{value}'...")
        # Setze den Wert im __dict__ der Instanz
        instance.__dict__[self.name] = value

class Person:
    # Wir weisen Instanzen unseres Deskriptors als Klassenattribute zu
    vorname = LoggingDescriptor("vorname")
    nachname = LoggingDescriptor("nachname")

    def __init__(self, vorname, nachname):
        # Der __init__ löst den __set__-Aufruf des Deskriptors aus
        self.vorname = vorname
        self.nachname = nachname

# Testen
p = Person("Anna", "Schmidt")
print("-" * 20)

# Dieser Lesezugriff löst den __get__-Aufruf des Deskriptors aus
print(f"Name: {p.vorname} {p.nachname}")
print("-" * 20)

# Dieser Schreibzugriff löst den __set__-Aufruf aus
p.nachname = "Müller"
print(f"Neuer Name: {p.nachname}")



[LOG] Schreibzugriff auf 'vorname' mit Wert 'Anna'...
[LOG] Schreibzugriff auf 'nachname' mit Wert 'Schmidt'...
--------------------
[LOG] Lesezugriff auf 'vorname'...
[LOG] Lesezugriff auf 'nachname'...
Name: Anna Schmidt
--------------------
[LOG] Schreibzugriff auf 'nachname' mit Wert 'Müller'...
[LOG] Lesezugriff auf 'nachname'...
Neuer Name: Müller



---

### **Beispiel B : Ein wiederverwendbarer `PositiveNumber`-Validierungs-Deskriptor**

Hier zeigen wir die wahre Stärke von Deskriptoren: die Erstellung wiederverwendbarer Komponenten für die Geschäftslogik. Dieser Deskriptor stellt sicher, dass ein Attribut immer eine positive Zahl ist.


In [3]:


class PositiveNumber:
    def __init__(self):
        # Wir verwenden einen "privaten" Namen, um den Wert im __dict__ der Instanz
        # zu speichern und Namenskollisionen zu vermeiden.
        self.private_name = None

    def __set_name__(self, owner, name):
        # Diese spezielle Methode (ab Python 3.6) speichert den Namen des Attributs,
        # dem der Deskriptor zugewiesen wird (z.B. 'rechnungsbetrag').
        self.private_name = '_' + name
        
    def __get__(self, instance, owner):
        return getattr(instance, self.private_name)

    def __set__(self, instance, value):
        if not isinstance(value, (int, float)):
            raise TypeError("Wert muss eine Zahl sein.")
        if value <= 0:
            raise ValueError("Wert muss positiv sein.")
        
        # Speichert den validierten Wert
        setattr(instance, self.private_name, value)

class Rechnung:
    # Wir wenden DENSELBEN Deskriptor auf MEHRERE Attribute an.
    rechnungsbetrag = PositiveNumber()
    artikelanzahl = PositiveNumber()
    
    def __init__(self, betrag, anzahl):
        # Löst den __set__-Aufruf des Deskriptors aus
        self.rechnungsbetrag = betrag
        self.artikelanzahl = anzahl

# Gültige Rechnung erstellen
r1 = Rechnung(199.99, 5)
print(f"Betrag: {r1.rechnungsbetrag}, Anzahl: {r1.artikelanzahl}")

# Versuch, ungültige Werte zuzuweisen
try:
    r1.rechnungsbetrag = -50
except ValueError as e:
    print(f"\nFehler bei Betrag: {e}")

try:
    r1.artikelanzahl = "drei"
except TypeError as e:
    print(f"Fehler bei Anzahl: {e}")



Betrag: 199.99, Anzahl: 5

Fehler bei Betrag: Wert muss positiv sein.
Fehler bei Anzahl: Wert muss eine Zahl sein.



---

### **6.1.2 Transfer auf das `Raumschiff`-Projekt**

Wir ersetzen nun unsere `@property`-Implementierung für ein Attribut durch einen eigenen, maßgeschneiderten Deskriptor. Damit beweisen wir, dass wir die Low-Level-Mechanik, die `@property` erst ermöglicht, vollständig verstanden haben.

Wir erstellen einen `EnergieLevelDescriptor`, der sicherstellt, dass der Energiewert immer in einem gültigen Bereich liegt.


In [4]:


class EnergieLevelDescriptor:
    def __init__(self, max_wert):
        self.max_wert = max_wert
        self.private_name = None

    def __set_name__(self, owner, name):
        self.private_name = '_' + name
        
    def __get__(self, instance, owner):
        # Gibt den Wert zurück, falls er existiert, sonst 0
        return getattr(instance, self.private_name, 0)

    def __set__(self, instance, value):
        print(f"-> Energie-Deskriptor: Setze Wert auf {value}...")
        if value > self.max_wert:
            print(f"   Wert zu hoch. Begrenze auf Maximum ({self.max_wert}).")
            setattr(instance, self.private_name, self.max_wert)
        elif value < 0:
            print(f"   Wert zu niedrig. Begrenze auf Minimum (0).")
            setattr(instance, self.private_name, 0)
        else:
            setattr(instance, self.private_name, value)
            
class Raumschiff:
    # Wir ersetzen die @property durch eine Instanz unseres Deskriptors
    energie_level = EnergieLevelDescriptor(max_wert=120)

    def __init__(self, name, start_energie):
        self.name = name
        self.energie_level = start_energie # Löst den __set__-Aufruf aus

    def __str__(self):
        return f"'{self.name}' [Energie: {self.energie_level}]"

# Testen
enterprise = Raumschiff("Enterprise", 100)
print(enterprise)

print("\nLade Energie über das Maximum...")
enterprise.energie_level = 150
print(enterprise)

print("\nVerbrauche zu viel Energie...")
enterprise.energie_level -= 200
print(enterprise)



-> Energie-Deskriptor: Setze Wert auf 100...
'Enterprise' [Energie: 100]

Lade Energie über das Maximum...
-> Energie-Deskriptor: Setze Wert auf 150...
   Wert zu hoch. Begrenze auf Maximum (120).
'Enterprise' [Energie: 120]

Verbrauche zu viel Energie...
-> Energie-Deskriptor: Setze Wert auf -80...
   Wert zu niedrig. Begrenze auf Minimum (0).
'Enterprise' [Energie: 0]



**Fazit:** Wir haben die Funktionalität von `@property` erfolgreich mit einem eigenen, wiederverwendbaren Deskriptor nachgebaut und damit die "Magie" dahinter vollständig entschlüsselt.

## **6.2: Metaprogrammierung – Klassen, die Klassen erschaffen**

Willkommen zum tiefsten Teil des Kaninchenbaus. In diesem Kapitel beschäftigen wir uns mit **Metaprogrammierung**: Code, der anderen Code zur Laufzeit analysiert, generiert oder modifiziert. Speziell schauen wir uns **Metaklassen** an – ein fortgeschrittenes Feature, das Ihnen die Kontrolle über den Prozess der Klassenerstellung selbst gibt.

---

### **Das fundamentale Konzept – In Python sind Klassen auch nur Objekte**

Um Metaklassen zu verstehen, müssen wir eine grundlegende Idee von Python vollständig verinnerlichen: **Alles in Python ist ein Objekt.**

* Eine Zahl ist ein Objekt der Klasse `int`.
* Ein String ist ein Objekt der Klasse `str`.
* Eine Liste ist ein Objekt der Klasse `list`.
* Eine Funktion ist ein Objekt der Klasse `function`.

Die logische Schlussfolgerung daraus ist: **Eine Klasse selbst muss auch ein Objekt sein.** Aber ein Objekt von welcher Klasse?

Die "Klasse einer Klasse" nennt man ihre **Metaklasse**.

---

### **Beispiel 1 : Die Klasse einer Klasse finden**

Wir nutzen die uns bekannte `type()`-Funktion, um das zu beweisen.


In [5]:


# Die 'Klasse' einer Ganzzahl ist 'int'
print(f"Der Typ von 5 ist: {type(5)}")

# Die 'Klasse' einer Liste ist 'list'
print(f"Der Typ von [1, 2] ist: {type([1, 2])}")

# Was ist die 'Klasse' einer Klasse?
class MeineKlasse:
    pass

# Die 'Klasse' der Klasse 'MeineKlasse' ist 'type'
print(f"Der Typ von MeineKlasse ist: {type(MeineKlasse)}")



Der Typ von 5 ist: <class 'int'>
Der Typ von [1, 2] ist: <class 'list'>
Der Typ von MeineKlasse ist: <class 'type'>



**Erkenntnis:** `type` ist die Standard-**Metaklasse** in Python. Jedes Mal, wenn Sie das `class`-Schlüsselwort verwenden, ruft Python im Hintergrund die Metaklasse `type` auf, um das Klassenobjekt für Sie zu erstellen.

---

### **`type()` als dynamische Klassen-Fabrik**

Die `type()`-Funktion hat eine zweite, weniger bekannte aber extrem mächtige Fähigkeit: Sie kann nicht nur den Typ eines Objekts zurückgeben, sondern auch **dynamisch neue Klassen zur Laufzeit erstellen**. Dies ist der erste Schritt der Metaprogrammierung.

Die Syntax dafür lautet: `type(name, bases, attrs)`

* `name`: Ein String, der der Name der neuen Klasse wird (z.B. `'Roboter'`).
* `bases`: Ein Tupel, das die Elternklassen enthält, von denen geerbt werden soll. Für eine Klasse ohne Eltern ist dies ein leeres Tupel `()`.
* `attrs`: Ein Dictionary, das die Attribute und Methoden der neuen Klasse enthält. Der Schlüssel ist der Attribut-/Methodenname (ein String), der Wert ist der Attributwert oder die Funktionsdefinition.

---

### **Beispiel 2 : Eine leere Klasse mit `type()` erstellen**


In [6]:


# Traditioneller Weg:
# class Roboter:
#     pass

# Dynamischer Weg mit type():
Roboter = type('Roboter', (), {})

print(f"Die neue Klasse: {Roboter}")
print(f"Der Typ der Klasse: {type(Roboter)}")

# Wir können davon ganz normal Instanzen erstellen
r1 = Roboter()
print(f"Eine Instanz davon: {r1}")



Die neue Klasse: <class '__main__.Roboter'>
Der Typ der Klasse: <class 'type'>
Eine Instanz davon: <__main__.Roboter object at 0x11cf95010>



---
### **Beispiel 3 : Eine Klasse mit Attributen dynamisch erstellen**


In [7]:


# Der 'attrs'-Dictionary enthält die Klassenattribute
roboter_attribute = {
    'hersteller': 'Boston Dynamics',
    'version': 2.1
}

RoboterMitAttributen = type('RoboterMitAttributen', (), roboter_attribute)

# Zugriff auf die Klassenattribute
print(f"Hersteller: {RoboterMitAttributen.hersteller}")
print(f"Version: {RoboterMitAttributen.version}")



Hersteller: Boston Dynamics
Version: 2.1



---

### **Beispiel 4 : Eine Klasse mit Methoden dynamisch erstellen**

Methoden sind auch nur Funktionen, die im `attrs`-Dictionary gespeichert werden.


In [8]:


# Schritt 1: Wir definieren eine normale Funktion.
# Sie muss 'self' als ersten Parameter akzeptieren.
def roboter_meldet_sich(self):
    print(f"Roboter {self.seriennummer} meldet sich zum Dienst.")

# Schritt 2: Wir definieren den Konstruktor
def roboter_init(self, seriennummer):
    self.seriennummer = seriennummer

# Schritt 3: Wir erstellen das 'attrs'-Dictionary
roboter_methoden = {
    '__init__': roboter_init,
    'melde_dich': roboter_meldet_sich
}

# Schritt 4: Wir bauen die Klasse dynamisch zusammen
KampfRoboter = type('KampfRoboter', (), roboter_methoden)

# Testen der dynamisch erstellten Klasse
k_roboter = KampfRoboter("X-101")
k_roboter.melde_dich() # Funktioniert!



Roboter X-101 meldet sich zum Dienst.



---

### **Eigene Metaklassen – Die Klassenerstellung manipulieren**

Wir wissen jetzt, dass `class ...` eine Abkürzung für einen `type(...)`-Aufruf ist. Was aber, wenn wir den Prozess der Klassenerstellung selbst **modifizieren oder kontrollieren** wollen? Hier kommen eigene Metaklassen ins Spiel.

**Warum sollte man das je tun?**
* **Automatische Registrierung:** Jede neu definierte Klasse (z.B. ein Plugin, ein Web-Controller, ein Datenbank-Modell) soll sich automatisch in einem zentralen Register eintragen, ohne dass der Entwickler einen extra `register()`-Aufruf machen muss.
* **Automatische Modifikation:** Jede Klasse soll automatisch um bestimmte Attribute oder Methoden erweitert werden (z.B. ein `erstellt_am`-Zeitstempel).
* **Validierung:** Wir wollen erzwingen, dass jede Klasse bestimmte Regeln einhält (z.B. alle Attributnamen müssen in Kleinbuchstaben sein).

**Wie funktioniert das?**
1.  Wir erstellen eine Klasse, die von `type` erbt.
2.  Wir überschreiben ihre `__new__`- oder `__init__`-Methode.
3.  Die `__new__(mcs, name, bases, attrs)`-Methode ist am mächtigsten. Sie wird aufgerufen, *bevor* das neue Klassenobjekt erstellt wird. Sie erhält:
    * `mcs`: Die Metaklasse selbst.
    * `name`, `bases`, `attrs`: Genau die Argumente, die auch `type()` erhält.
4.  Innerhalb von `__new__` können wir `name`, `bases` oder `attrs` manipulieren.
5.  Am Ende rufen wir `super().__new__(mcs, name, bases, attrs)` auf, um das eigentliche Klassenobjekt zu erstellen und zurückzugeben.

---

### **Beispiel 5 : Eine `LoggingMeta`**

Diese Metaklasse tut nichts weiter, als bei der Erstellung einer Klasse deren Details auszugeben.


In [9]:


class LoggingMeta(type):
    # __new__ wird aufgerufen, wenn eine Klasse mit dieser Metaklasse DEFINIERT wird.
    def __new__(mcs, name, bases, attrs):
        print(f"--- Metaklasse '{mcs.__name__}' wird aktiv! ---")
        print(f"Erstelle Klasse namens: '{name}'")
        print(f"Elternklassen (Bases): {bases}")
        print(f"Attribute & Methoden (Attrs): {attrs}")
        
        # Erstelle das eigentliche Klassenobjekt
        neue_klasse = super().__new__(mcs, name, bases, attrs)
        print("--- Klasse wurde erstellt. ---")
        return neue_klasse

# Wir wenden die Metaklasse mit dem 'metaclass'-Keyword an
# Die print-Ausgaben erscheinen jetzt, bei der Definition, nicht bei der Instanziierung!
print("Definiere jetzt die Spieler-Klasse...")
class Spieler(metaclass=LoggingMeta):
    hp = 100
    
    def angreifen(self):
        print("Angriff!")
print("Definition der Spieler-Klasse abgeschlossen.")

# Die Erstellung einer Instanz ist ein normaler Vorgang und löst die Metaklasse nicht erneut aus.
print("\nErstelle jetzt eine Spieler-Instanz...")
s1 = Spieler()



Definiere jetzt die Spieler-Klasse...
--- Metaklasse 'LoggingMeta' wird aktiv! ---
Erstelle Klasse namens: 'Spieler'
Elternklassen (Bases): ()
Attribute & Methoden (Attrs): {'__module__': '__main__', '__qualname__': 'Spieler', '__firstlineno__': 17, 'hp': 100, 'angreifen': <function Spieler.angreifen at 0x11cfc0360>, '__static_attributes__': ()}
--- Klasse wurde erstellt. ---
Definition der Spieler-Klasse abgeschlossen.

Erstelle jetzt eine Spieler-Instanz...



---

### **Beispiel 6 : Eine Metaklasse, die Attribute erzwingt**

Diese Metaklasse validiert bei der Klassendefinition, ob ein bestimmtes Klassenattribut vorhanden ist.


In [10]:


class IDZwingendMeta(type):
    def __new__(mcs, name, bases, attrs):
        print(f"Prüfe Klasse '{name}' auf 'klassen_id'...")
        # Validierungslogik: Ist der Schlüssel 'klassen_id' nicht im Attribut-Dictionary?
        if 'klassen_id' not in attrs:
            # Wenn nicht, löse einen Fehler aus und verhindere die Klassenerstellung.
            raise TypeError(f"Die Klasse '{name}' muss ein 'klassen_id'-Attribut definieren!")
        
        print(f"'{name}' ist gültig.")
        return super().__new__(mcs, name, bases, attrs)

# Dieser Versuch wird funktionieren
class GueltigesModul(metaclass=IDZwingendMeta):
    klassen_id = "MOD-A"
    version = "1.0"
    
# Dieser Versuch wird einen TypeError auslösen, weil 'klassen_id' fehlt
try:
    class UngueltigesModul(metaclass=IDZwingendMeta):
        version = "2.0"
except TypeError as e:
    print(f"\nFEHLER wie erwartet: {e}")



Prüfe Klasse 'GueltigesModul' auf 'klassen_id'...
'GueltigesModul' ist gültig.
Prüfe Klasse 'UngueltigesModul' auf 'klassen_id'...

FEHLER wie erwartet: Die Klasse 'UngueltigesModul' muss ein 'klassen_id'-Attribut definieren!



---
---

### **6.2.2 Transfer auf das `Raumschiff`-Projekt: Das automatische Schiffsregister**

Wir erstellen eine Metaklasse `SchiffsRegisterMeta`. Jedes Mal, wenn eine neue `Schiffsklasse` (wie `Fregatte` oder `Zerstörer`) definiert wird, soll sie sich automatisch in einem zentralen Register eintragen, ohne dass wir manuell Code dafür schreiben müssen.


In [11]:


class SchiffsRegisterMeta(type):
    # Ein Dictionary auf der Metaklasse selbst, um alle Schiffstypen zu speichern
    _register = {}

    def __new__(mcs, name, bases, attrs):
        # Wir erstellen zuerst ganz normal die neue Klasse
        neue_klasse = super().__new__(mcs, name, bases, attrs)
        
        # Nach der Erstellung registrieren wir sie, aber nur, wenn es keine Basisklasse ist
        if name != 'Schiffsklasse':
            print(f"[REGISTER] Neue Schiffsklasse '{name}' wird automatisch registriert.")
            mcs._register[name.lower()] = neue_klasse
            
        return neue_klasse

# Unsere Basisklasse verwendet die Metaklasse
class Schiffsklasse(metaclass=SchiffsRegisterMeta):
    def __init__(self, name):
        self.name = name

# Sobald Python diese Zeilen liest, wird die Metaklasse aktiv!
class Fregatte(Schiffsklasse):
    pass

class Zerstörer(Schiffsklasse):
    pass

class Korvette(Schiffsklasse):
    pass

print("\n--- Schiffs-Definitionen abgeschlossen ---")

# Jetzt können wir auf das Register der Metaklasse zugreifen
print("\nVerfügbare Schiffstypen im Register:")
print(SchiffsRegisterMeta._register)

# Wir können sogar eine Fabrik bauen, die Schiffe aus dem Register erstellt
def baue_schiff(typ, name):

_IncompleteInputError: incomplete input (153784155.py, line 38)

## **6.3: Gesamtwiederholung & Prüfungssimulation**

Willkommen zum finalen Abschnitt unseres OOP-Kurses. Wir haben eine lange und tiefe Reise durch das Python-Objektmodell hinter uns. In diesem letzten Teil fassen wir alle zentralen Konzepte noch einmal zusammen und stellen uns dann komplexen Herausforderungen im Stil der PCPP-1-Prüfung, bei denen wir unser gesamtes Wissen kombinieren müssen.

---

### **6.3.1 Theorie & Praxis: Die große Wiederholung**

Lassen Sie uns einen schnellen, aber umfassenden Überblick über die Konzepte der Woche geben. Dies ist Ihre Checkliste für die Prüfung.

* **Grundprinzipien (Tag 1):**
    * **Klasse & Instanz:** Der Bauplan und das daraus erstellte, eigenständige Objekt.
    * **`__init__` & `self`:** Der Konstruktor, der jede Instanz initialisiert, und die Referenz auf die Instanz selbst.
    * **Attribute (Instanz vs. Klasse):** Individuelle Daten pro Objekt (`self.x`) vs. geteilte Daten für alle Objekte der Klasse (`Klasse.x`).
    * **`__dict__`:** Der "Lagerraum", der die Attribute einer Instanz bzw. Klasse speichert.

* **Vererbung & Hierarchien (Tag 2 & 3):**
    * **Einfache & Mehrfachvererbung:** Das "is a"-Prinzip und das Kombinieren von Fähigkeiten ("Mixins").
    * **`super()`:** Die Brücke zur Elternklasse, um deren Funktionalität aufzurufen und zu erweitern.
    * **MRO (Method Resolution Order):** Pythons exakte Regel, um bei Mehrfachvererbung die richtige Methode in der Hierarchie zu finden.
    * **Polymorphismus & Duck Typing:** Objekte verschiedener Klassen über die gleiche Schnittstelle ansprechen. Es zählt, was ein Objekt *kann*, nicht, was es *ist*.
    * **Overriding:** Das Spezialisieren von geerbten Methoden in der Kindklasse.

* **Dekoratoren & Klassendesign (Tag 4):**
    * **Dekoratoren von Grund auf:** Funktionen als Objekte, Wrapper-Funktionen und die `@`-Syntax als syntaktischer Zucker.
    * **`@property`:** Der "Pythonic Way", um Getter und Setter für einen kontrollierten, sauberen Attributzugriff zu implementieren.
    * **`@classmethod`:** Methoden, die auf der Klasse (`cls`) statt der Instanz arbeiten, primär für alternative Konstruktoren.
    * **`@staticmethod`:** Normale Funktionen, die im Namensraum einer Klasse "geparkt" sind, um die Code-Struktur zu verbessern.

* **Operationen, Abstraktion & Container (Tag 5):**
    * **Magic Methods für Operationen:** `__eq__`, `__lt__` für Vergleiche und `__add__`, `__mul__` für Arithmetik.
    * **Abstrakte Basisklassen (ABCs):** Mit `@abstractmethod` einen "Vertrag" für Kindklassen definieren, um eine konsistente Schnittstelle zu erzwingen.
    * **Von Built-ins erben:** Klassen wie `list` oder `dict` als Basis für eigene, spezialisierte Typen nutzen.
    * **Container- & Iterator-Protokoll:** `__len__`, `__getitem__` und vor allem `__iter__` / `__next__`, um eigene Objekte iterierbar zu machen.

* **Objektzustand & Fortgeschrittene Konzepte (Tag 6 & 7):**
    * **Kopieren:** Der kritische Unterschied zwischen einer flachen (`copy`) und einer tiefen (`deepcopy`) Kopie.
    * **Serialisierung:** Objekte mit `pickle` und `shelve` speichern und laden.
    * **Exceptions:** Eigene Fehlerklassen definieren und mit `raise from` verketten.
    * **Deskriptoren:** Das Low-Level-Protokoll (`__get__`, `__set__`), das `@property` seine Magie verleiht.
    * **Metaklassen:** Das fortgeschrittenste Konzept – Klassen, die Klassen zur Laufzeit erzeugen und modifizieren.

---
---

### **6.3.2 Prüfungssimulations-Aufgaben**

Die folgenden Aufgaben sind keine einfachen Übungen, sondern komplexe Herausforderungen. Sie erfordern die Kombination mehrerer der oben genannten Konzepte, um eine vollständige Lösung zu entwickeln – genau wie in der PCPP-1-Prüfung.

---
**Beispiel-Challenge 1: Die versionierte, serialisierbare & validierte `Config`-Klasse**

**Aufgabe:**
Erstellen Sie eine `Config`-Klasse, die Konfigurationseinstellungen für eine Anwendung verwaltet. Sie muss folgende Anforderungen erfüllen:
1.  **Versionierbar & Sortierbar:** Jede `Config`-Instanz hat eine `version` (z.B. `(1, 2, 0)`). Eine Liste von Config-Objekten soll nach dieser Version sortierbar sein.
2.  **Typ-sicher:** Die Attribute `autor` (muss ein `str` sein) und `max_verbindungen` (muss ein `int` sein) sollen bei der Zuweisung validiert werden. Nutzen Sie dafür einen wiederverwendbaren **Deskriptor**.
3.  **Serialisierbar:** Instanzen der Klasse müssen mit `pickle` gespeichert und geladen werden können.


In [ ]:


# --- LÖSUNG FÜR CHALLENGE 1 ---
import pickle
from functools import total_ordering

# Schritt 1: Der wiederverwendbare Validierungs-Deskriptor
class TypValidierungsDescriptor:
    def __init__(self, erwarteter_typ):
        self.erwarteter_typ = erwarteter_typ
        self.private_name = None

    def __set_name__(self, owner, name):
        self.private_name = '_' + name

    def __get__(self, instance, owner):
        return getattr(instance, self.private_name)

    def __set__(self, instance, value):
        if not isinstance(value, self.erwarteter_typ):
            raise TypeError(f"'{self.private_name[1:]}' muss vom Typ {self.erwarteter_typ.__name__} sein.")
        setattr(instance, self.private_name, value)

# Schritt 2: Die Config-Klasse, die alles kombiniert
@total_ordering
class Config:
    # Schritt 3: Deskriptoren für typsichere Attribute anwenden
    autor = TypValidierungsDescriptor(str)
    max_verbindungen = TypValidierungsDescriptor(int)

    def __init__(self, version_string, autor, max_verbindungen):
        self.version = tuple(map(int, version_string.split('.')))
        self.autor = autor # Löst den Deskriptor __set__ aus
        self.max_verbindungen = max_verbindungen # Löst den Deskriptor __set__ aus

    # Schritt 4: Vergleichsmethoden für Sortierbarkeit
    def __eq__(self, other):
        if not isinstance(other, Config): return NotImplemented
        return self.version == other.version

    def __lt__(self, other):
        if not isinstance(other, Config): return NotImplemented
        return self.version < other.version
        
    def __repr__(self):
        return f"Config(Version {'.'.join(map(str, self.version))}, Autor: {self.autor})"

# --- Test-Szenario ---
print("--- TEST CHALLENGE 1 ---")
# Erstellen von Instanzen
c1 = Config("1.2.0", "Anna", 100)
c2 = Config("1.10.0", "Ben", 200)
c3 = Config("0.9.5", "Carla", 50)

# Test der Validierung durch Deskriptoren
try:
    c1.max_verbindungen = "unbegrenzt"
except TypeError as e:
    print(f"Fehler wie erwartet: {e}")

# Test der Sortierbarkeit
configs = [c1, c2, c3]
configs.sort()
print(f"\nSortierte Konfigurationen: {configs}")

# Test der Serialisierung (Pickle)
print("\nSpeichere und lade Konfiguration c2...")
with open("config.pkl", "wb") as f:
    pickle.dump(c2, f)

with open("config.pkl", "rb") as f:
    geladene_config = pickle.load(f)

print(f"Geladenes Objekt: {geladene_config}")
print(f"Vergleich mit Original: {geladene_config == c2}")





---
**Beispiel-Challenge 2: Die abstrakte, iterierbare `DatenPipeline` mit Dekorator**

**Aufgabe:**
Erstellen Sie ein kleines Framework für eine Datenpipeline.
1.  **Abstrakte Schnittstelle:** Eine ABC `Datenquelle` soll vorschreiben, dass jede Quelle eine `lesen()`-Methode haben muss.
2.  **Konkrete Quellen:** Implementieren Sie `DateiQuelle` und `NetzwerkQuelle`, die diese Schnittstelle erfüllen.
3.  **Dekorator:** Ein Dekorator `@log_verarbeitung` soll den Start und das Ende einer Methode protokollieren.
4.  **Iterable Pipeline:** Eine Klasse `DatenPipeline`, die eine Liste von `Datenquelle`-Objekten entgegennimmt und über die man mit einer `for`-Schleife iterieren kann. Ihre Hauptmethode `verarbeiten()` soll dekoriert sein und alle Quellen nacheinander auslesen.


In [ ]:


# --- LÖSUNG FÜR CHALLENGE 2 ---
from abc import ABC, abstractmethod
import time

# Schritt 1: Der Dekorator
def log_verarbeitung(func):
    def wrapper(*args, **kwargs):
        print("--- [Pipeline] Verarbeitung wird gestartet... ---")
        start_zeit = time.time()
        ergebnis = func(*args, **kwargs)
        end_zeit = time.time()
        print(f"--- [Pipeline] Verarbeitung abgeschlossen in {end_zeit - start_zeit:.2f}s. ---")
        return ergebnis
    return wrapper

# Schritt 2: Die Abstrakte Basisklasse
class Datenquelle(ABC):
    @abstractmethod
    def lesen(self):
        pass

# Schritt 3: Die konkreten Klassen
class DateiQuelle(Datenquelle):
    def __init__(self, dateipfad):
        self.dateipfad = dateipfad
    def lesen(self):
        print(f"Lese Daten aus Datei: '{self.dateipfad}'")
        return f"Inhalt von {self.dateipfad}"

class NetzwerkQuelle(Datenquelle):
    def __init__(self, url):
        self.url = url
    def lesen(self):
        print(f"Lade Daten von URL: '{self.url}'")
        return f"Response von {self.url}"

# Schritt 4: Der iterierbare Container
class DatenPipeline:
    def __init__(self, quellen):
        # Sicherstellen, dass alle Quellen gültig sind
        self._quellen = [q for q in quellen if isinstance(q, Datenquelle)]
        self._index = 0

    # Iterator-Protokoll
    def __iter__(self):
        self._index = 0
        return self
    
    def __next__(self):
        if self._index < len(self._quellen):
            quelle = self._quellen[self._index]
            self._index += 1
            return quelle
        else:
            raise StopIteration
            
    @log_verarbeitung
    def verarbeite_alle(self):
        gesammelte_daten = []
        # Die Pipeline ist dank __iter__/__next__ selbst iterierbar
        for quelle in self:
            daten = quelle.lesen()
            gesammelte_daten.append(daten.upper()) # Simulation einer Verarbeitung
            time.sleep(0.5)
        return gesammelte_daten

# --- Test-Szenario ---
print("--- TEST CHALLENGE 2 ---")
# Quellen erstellen
q1 = DateiQuelle("report.csv")
q2 = NetzwerkQuelle("[api.example.com/data](https://api.example.com/data)")
q3 = DateiQuelle("log.txt")

# Pipeline erstellen
pipeline = DatenPipeline([q1, q2, q3])

# Die dekorierte Methode aufrufen
ergebnisse = pipeline.verarbeite_alle()
print(f"\nVerarbeitete Ergebnisse: {ergebnisse}")
